In [27]:
# this will convert the AmnestyQA dataset
# https://huggingface.co/datasets/explodinggradients/amnesty_qa/blob/main/english.json
# to the desired format that we are using in the competition
# https://github.com/sujitpal/llm-rag-eval
#
# download the above json file to a file local to this notebook
# (i had trouble reading this directly online - https://huggingface.co/datasets/explodinggradients/amnesty_qa/raw/main/english.json)

In [28]:
import json
# input file
input_file_path = 'english.json'
# output file
output_file_path = 'amnesty_qa.jsonl'

In [29]:
def split_newlines(input_string):
    """Split the input string at newline characters and return a list of strings."""
    if '\n' in input_string:
        return input_string.split('\n')
    else:
        return [input_string]

In [31]:
# Opening the JSON file and loading the data
with open(input_file_path, 'r') as file:
    data = json.load(file)

In [32]:
# the data is not nested / folded
# so each of components is read in and the output folding happens next
questions = data.get('question')
ground_truths = data.get('ground_truths')
answers = data.get('answer')
contexts = data.get('contexts')

In [33]:
# output will be lines
lines = []

In [34]:
# assemble from the nth item from each component list
# blow out the multi-line contexts into single line chunk dictionaries
for i in range(len(questions)):
  line = {}
  line['id'] = i
  line['query'] = questions[i]
  context_list = []
  split_contexts = split_newlines(contexts[i][0])
  for j in range(len(split_contexts)):
    context_dict = {}
    context_dict['id'] = str(j)
    context_dict['chunk_text'] = split_contexts[j]
    context_list.append(context_dict)
  line['context'] = context_list
  line['ideal_answer'] = ground_truths[i][0]
  line['predicted_answer'] = answers[i]
  lines.append(line)

In [36]:
# write the lines to the output file
with open(output_file_path, 'w') as outfile:
    for dictionary in lines:
        json_line = json.dumps(dictionary)  # Convert the dictionary to a JSON string
        outfile.write(json_line + '\n')  # Write the JSON string to the file and add a newline